# AI-Driven Customer Churn Prediction & Segmentation
## Phase 1 — Exploratory Data Analysis (EDA) & KPI Baseline

**Goal:** Understand churn patterns and build baseline KPIs before modeling.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 200)

df = pd.read_csv("../data/raw/telco_churn.csv")
df.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [2]:
import os
os.getcwd()



'C:\\Users\\oumayma\\anaconda_projects\\9f39a66f-b44d-4769-9ccd-532a921e489a\\churn-ai-segmentation\\notebooks'

In [3]:
os.listdir("../data/raw")


['.ipynb_checkpoints', 'telco_churn.csv']

In [5]:
import os
os.listdir("../data/raw")


['.ipynb_checkpoints', 'telco_churn.csv']

In [7]:
df = pd.read_csv("../data/raw/telco_churn.csv")



In [8]:
df.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [9]:
df.shape, df.dtypes


((7043, 21),
 customerID           object
 gender               object
 SeniorCitizen         int64
 Partner              object
 Dependents           object
 tenure                int64
 PhoneService         object
 MultipleLines        object
 InternetService      object
 OnlineSecurity       object
 OnlineBackup         object
 DeviceProtection     object
 TechSupport          object
 StreamingTV          object
 StreamingMovies      object
 Contract             object
 PaperlessBilling     object
 PaymentMethod        object
 MonthlyCharges      float64
 TotalCharges         object
 Churn                object
 dtype: object)

## Data cleaning & target preparation
We clean critical columns and encode the churn variable to prepare the dataset for KPI analysis and modeling.


In [10]:
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")
df.isna().sum().sort_values(ascending=False).head(10)


TotalCharges        11
customerID           0
DeviceProtection     0
MonthlyCharges       0
PaymentMethod        0
PaperlessBilling     0
Contract             0
StreamingMovies      0
StreamingTV          0
TechSupport          0
dtype: int64

In [11]:
df = df.dropna(subset=["TotalCharges"]).copy()
df["Churn"] = df["Churn"].map({"Yes": 1, "No": 0})
df["Churn"].value_counts(normalize=True)


Churn
0    0.734215
1    0.265785
Name: proportion, dtype: float64

In [12]:
df.shape


(7032, 21)

In [13]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                 int64
dtype: object

## Churn breakdown by key business drivers
We compare churn rates across key customer attributes (contract type, tenure, and monthly charges). 
This helps identify early churn drivers and informs what features will matter in the model.


In [16]:
df.groupby("Contract")["Churn"].agg(
    churn_rate="mean",
    customers="count"
)


,churn_rate,customers
Contract,,
Month-to-month,0.427097,3875
One year,0.112772,1472
Two year,0.028487,1685


In [18]:
def churn_table(col):
    return (
        df.groupby(col)["Churn"]
          .agg(churn_rate="mean", customers="count")
          .sort_values("churn_rate", ascending=False)
    )


In [19]:
df["tenure_group"] = pd.cut(
    df["tenure"],
    bins=[0, 6, 12, 24, 48, 72],
    labels=["0-6m", "7-12m", "13-24m", "25-48m", "49-72m"],
    include_lowest=True
)

tenure_tbl = churn_table("tenure_group")
tenure_tbl


C:\Users\oumayma\AppData\Local\Temp\ipykernel_34056\3405914196.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(col)["Churn"]


,churn_rate,customers
tenure_group,,
0-6m,0.533333,1470
7-12m,0.358865,705
13-24m,0.287109,1024
25-48m,0.203890,1594
49-72m,0.095132,2239


In [20]:
df["charges_group"] = pd.qcut(df["MonthlyCharges"], q=4, duplicates="drop")

charges_tbl = df.groupby("charges_group")["Churn"].agg(
    churn_rate="mean",
    customers="count"
).sort_values("charges_group")

charges_tbl


C:\Users\oumayma\AppData\Local\Temp\ipykernel_34056\1869967344.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  charges_tbl = df.groupby("charges_group")["Churn"].agg(


,churn_rate,customers
charges_group,,
"(18.249, 35.588]",0.113197,1758
"(35.588, 70.35]",0.245883,1761
"(70.35, 89.862]",0.375499,1755
"(89.862, 118.75]",0.328783,1758
